In [ ]:
!sudo apt update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
#Check this site for the latest download link https://www.apache.org/dyn/closer.lua/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz
!wget -q https://dlcdn.apache.org/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz
!tar xf spark-3.2.1-bin-hadoop3.2.tgz
!pip install -q findspark
!pip install pyspark
!pip install py4j

import os
import sys
# os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
# os.environ["SPARK_HOME"] = "/content/spark-3.2.1-bin-hadoop3.2"


import findspark
findspark.init()
findspark.find()

import pyspark

from pyspark.sql import DataFrame, SparkSession
from typing import List
import pyspark.sql.types as T
import pyspark.sql.functions as F

spark= SparkSession.builder.appName("Our First Spark Example").getOrCreate()

spark

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Hit:6 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:10 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:11 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [1,978 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/restricted amd64 Packages [2,644 kB]
Get:13 http://security.ubuntu.com/ubuntu jammy-sec

In [ ]:
data=[("Alice",25),("Bod",30),("Charlie",35)]
df=spark.createDataFrame(data,["Name","Age"])
df.show()

+-------+---+
|   Name|Age|
+-------+---+
|  Alice| 25|
|    Bod| 30|
|Charlie| 35|
+-------+---+



In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials


In [ ]:
auth.authenticate_user()
gauth=GoogleAuth()
gauth.credentials=GoogleCredentials.get_application_default()
drive=GoogleDrive(gauth)

In [ ]:
downloaded=drive.CreateFile({'id':"12D7fQjfQJIJn2OBgpLg5ZrNIYGvw6ksP"})
downloaded.GetContentFile('diabetes.csv')

In [ ]:
sc=spark.sparkContext
path="diabetes.csv"
df=spark.read.csv(path)
df.show()

+-----------+-------+-------------+-------------+-------+----+--------------------+---+-------+
|        _c0|    _c1|          _c2|          _c3|    _c4| _c5|                 _c6|_c7|    _c8|
+-----------+-------+-------------+-------------+-------+----+--------------------+---+-------+
|Pregnancies|Glucose|BloodPressure|SkinThickness|Insulin| BMI|DiabetesPedigreeF...|Age|Outcome|
|          6|    148|           72|           35|      0|33.6|               0.627| 50|      1|
|          1|     85|           66|           29|      0|26.6|               0.351| 31|      0|
|          8|    183|           64|            0|      0|23.3|               0.672| 32|      1|
|          1|     89|           66|           23|     94|28.1|               0.167| 21|      0|
|          0|    137|           40|           35|    168|43.1|               2.288| 33|      1|
|          5|    116|           74|            0|      0|25.6|               0.201| 30|      0|
|          3|     78|           50|     

In [ ]:
sc=spark.sparkContext
path="diabetes.csv"
df=spark.read.csv(path,header=True)
df.show()

+-----------+-------+-------------+-------------+-------+----+------------------------+---+-------+
|Pregnancies|Glucose|BloodPressure|SkinThickness|Insulin| BMI|DiabetesPedigreeFunction|Age|Outcome|
+-----------+-------+-------------+-------------+-------+----+------------------------+---+-------+
|          6|    148|           72|           35|      0|33.6|                   0.627| 50|      1|
|          1|     85|           66|           29|      0|26.6|                   0.351| 31|      0|
|          8|    183|           64|            0|      0|23.3|                   0.672| 32|      1|
|          1|     89|           66|           23|     94|28.1|                   0.167| 21|      0|
|          0|    137|           40|           35|    168|43.1|                   2.288| 33|      1|
|          5|    116|           74|            0|      0|25.6|                   0.201| 30|      0|
|          3|     78|           50|           32|     88|  31|                   0.248| 26|      1|


In [ ]:
type(df)

pyspark.sql.dataframe.DataFrame

In [ ]:
dfrdd=df.rdd
type(dfrdd)

pyspark.rdd.RDD

In [ ]:
dfrdd.map(lambda c:(c[0],c[1],c[2],c[3])).take(4)


[('6', '148', '72', '35'),
 ('1', '85', '66', '29'),
 ('8', '183', '64', '0'),
 ('1', '89', '66', '23')]

In [ ]:
dfrdd.map(lambda c:(c[0],c[1],c[2],c[3])).filter(lambda c:c[3]=='35').take(5)


[('6', '148', '72', '35'),
 ('0', '137', '40', '35'),
 ('9', '119', '80', '35'),
 ('6', '148', '72', '35'),
 ('0', '137', '40', '35')]

In [ ]:
df.createOrReplaceTempView("diabeticview")
from pyspark.sql import SQLContext
sqlc=SQLContext(sc)
sqlc.sql("select * from diabeticview").show(10)

/usr/local/lib/python3.10/dist-packages/pyspark/sql/context.py:113: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


+-----------+-------+-------------+-------------+-------+----+------------------------+---+-------+
|Pregnancies|Glucose|BloodPressure|SkinThickness|Insulin| BMI|DiabetesPedigreeFunction|Age|Outcome|
+-----------+-------+-------------+-------------+-------+----+------------------------+---+-------+
|          6|    148|           72|           35|      0|33.6|                   0.627| 50|      1|
|          1|     85|           66|           29|      0|26.6|                   0.351| 31|      0|
|          8|    183|           64|            0|      0|23.3|                   0.672| 32|      1|
|          1|     89|           66|           23|     94|28.1|                   0.167| 21|      0|
|          0|    137|           40|           35|    168|43.1|                   2.288| 33|      1|
|          5|    116|           74|            0|      0|25.6|                   0.201| 30|      0|
|          3|     78|           50|           32|     88|  31|                   0.248| 26|      1|


In [ ]:
sqlc.sql("select Glucose,BloodPressure,SkinThickness from diabeticview").show(10)

+-------+-------------+-------------+
|Glucose|BloodPressure|SkinThickness|
+-------+-------------+-------------+
|    148|           72|           35|
|     85|           66|           29|
|    183|           64|            0|
|     89|           66|           23|
|    137|           40|           35|
|    116|           74|            0|
|     78|           50|           32|
|    115|            0|            0|
|    197|           70|           45|
|    125|           96|            0|
+-------+-------------+-------------+
only showing top 10 rows



In [ ]:
sqlc.sql("select * from diabeticview where Glucose>180 and BloodPressure>100").show(10)

+-----------+-------+-------------+-------------+-------+----+------------------------+---+-------+
|Pregnancies|Glucose|BloodPressure|SkinThickness|Insulin| BMI|DiabetesPedigreeFunction|Age|Outcome|
+-----------+-------+-------------+-------------+-------+----+------------------------+---+-------+
|          0|    189|          104|           25|      0|34.3|                   0.435| 41|      1|
|          4|    189|          110|           31|      0|28.5|                    0.68| 37|      0|
+-----------+-------+-------------+-------------+-------+----+------------------------+---+-------+



In [ ]:
sqlc.sql("select Outcome,count(*) from diabeticview group by Outcome").show()

+-------+--------+
|Outcome|count(1)|
+-------+--------+
|      0|     509|
|      1|     286|
+-------+--------+



In [ ]:
simpleData=[("james","sales",3000),
            ("michal","sales",4600),
            ("robert","sales",4100),
            ("maria","finance",3000),
            ("james","sales",3000),
            ("scout","finance",3300),
            ("jeff","marketing",3000),
            ("kumar","marketing",2000),
            ("safi","sales",4300)]
schema=["employee_name","department","salary"]
df=spark.createDataFrame(data=simpleData,schema=schema)
df.printSchema()
df.show(truncate=False)

root
 |-- employee_name: string (nullable = true)
 |-- department: string (nullable = true)
 |-- salary: long (nullable = true)

+-------------+----------+------+
|employee_name|department|salary|
+-------------+----------+------+
|james        |sales     |3000  |
|michal       |sales     |4600  |
|robert       |sales     |4100  |
|maria        |finance   |3000  |
|james        |sales     |3000  |
|scout        |finance   |3300  |
|jeff         |marketing |3000  |
|kumar        |marketing |2000  |
|safi         |sales     |4300  |
+-------------+----------+------+



In [ ]:
df.createOrReplaceTempView("empview")
from pyspark.sql import SQLContext
sqlc=SQLContext(sc)
sqlc.sql("select * from empview").show(10)

/usr/local/lib/python3.10/dist-packages/pyspark/sql/context.py:113: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


+-------------+----------+------+
|employee_name|department|salary|
+-------------+----------+------+
|        james|     sales|  3000|
|       michal|     sales|  4600|
|       robert|     sales|  4100|
|        maria|   finance|  3000|
|        james|     sales|  3000|
|        scout|   finance|  3300|
|         jeff| marketing|  3000|
|        kumar| marketing|  2000|
|         safi|     sales|  4300|
+-------------+----------+------+



In [ ]:
sqlc.sql("select department,sum(salary) from empview group by department").show()

+----------+-----------+
|department|sum(salary)|
+----------+-----------+
|   finance|       6300|
|     sales|      19000|
| marketing|       5000|
+----------+-----------+

